In [18]:
import pandas as pd
from scipy import linalg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import colors
from mpl_toolkits.mplot3d import Axes3D
from sklearn.qda import QDA
import random
pd.set_option('display.max_rows', 30)

In [19]:
#Settings
t_data_perc = 0.3333

In [20]:
#Read cancer data from the excel file
df = pd.read_excel('dataTable.xlsx', sheetname = 0, header = 0)
x1 = np.array(df['NADH_Intensity_B']).reshape(-1,1)
y1 = np.array(df['Fad_Intensity_G']).reshape(-1,1)
z1 = np.array(df['redoxRatioPix']).reshape(-1,1)
cancer_pt = np.hstack([x1, y1, z1])

#Read normal data from the excel file
df2 = pd.read_excel('dataTable.xlsx', sheetname = 1, header = 0)
x2 = np.array(df2['NADH_Intensity_B']).reshape(-1,1)
y2 = np.array(df2['Fad_Intensity_G']).reshape(-1,1)
z2 = np.array(df2['redoxRatioPix']).reshape(-1,1)
normal_pt = np.hstack([x2, y2, z2])

norDataNum, normal_ndim = normal_pt.shape
cnDataNum, cancer_ndim = cancer_pt.shape

#Sort given training data with corresponding labels
nor_n = np.zeros(int(normal_pt.size/normal_ndim))
can_n = np.ones(int(cancer_pt.size/cancer_ndim))
labels = np.hstack((nor_n, can_n))
train_data = np.vstack((normal_pt, cancer_pt))
normal_pt

array([[  6.17183420e+01,   9.10645398e+01,   5.27326164e-02],
       [  1.16681040e+02,   9.18781169e+01,   1.97783547e-02],
       [  1.22245786e+02,   9.81470863e+01,   2.28818258e-02],
       [  1.17923724e+02,   1.04429809e+02,   1.95357505e-02],
       [  6.78922932e+01,   7.52983957e+01,   3.66027385e-02],
       [  9.34843662e+01,   9.12943605e+01,   3.41144488e-02],
       [  8.51281884e+01,   7.87835367e+01,   2.80205943e-02],
       [  1.04633749e+02,   9.41237276e+01,   2.33339155e-02],
       [  1.06508016e+02,   9.71825584e+01,   1.19841283e-02],
       [  1.04088487e+02,   1.04756378e+02,   2.68098408e-02],
       [  7.02659419e+01,   7.47430068e+01,   2.60239956e-02],
       [  6.99665677e+01,   8.94889549e+01,   5.38412336e-02],
       [  6.04400945e+01,   5.43692314e+01,   3.03927802e-02],
       [  1.14079248e+02,   9.99329210e+01,   2.74550795e-02],
       [  1.03549168e+02,   9.24883686e+01,   2.56329827e-02],
       [  1.23374523e+02,   1.01794226e+02,   1.2947946

In [21]:
def randTestData(perc, size):
    
    if perc > 1:
        perc = 1
    
    smpSize = int(perc*size)
    labelTesting = np.ones(size) == 1
    smpIndice = random.sample(list(range(0, size)), smpSize)
    
    for i in smpIndice:
        labelTesting[i] = False
    
    return size-smpSize, labelTesting

In [22]:
def QDAResult3D():

    norTrainNum, nor_isTraining = randTestData(t_data_perc, norDataNum)
    cnTrainNum, cn_isTraining = randTestData(t_data_perc, cnDataNum)
    isTraining =np.hstack((nor_isTraining, cn_isTraining))

    #Training QDA classifier
    clf = QDA()
    trained_clf = clf.fit(train_data[isTraining], labels[isTraining])

     #Using the remaining data for testing
    normal_pred = trained_clf.predict(normal_pt[nor_isTraining == False])
    trueneg_n = (normal_pred == 0).sum()
    specificity = trueneg_n/int(norDataNum - norTrainNum)

    cancer_pred = trained_clf.predict(cancer_pt[cn_isTraining == False])
    truepos_n = (cancer_pred == 1).sum()
    sensitivity = truepos_n/int(cnDataNum - cnTrainNum)
    
    return sensitivity, specificity

In [23]:
result = np.zeros(40).reshape((20, 2))

for i in range(20):
    result[i] = QDAResult3D()

specAvr = result[:, 1].sum()/20
sp_min = result[:, 1].min()
sp_max = result[:, 1].max()
sensAvr = result[:, 0].sum()/20
sn_min = result[:, 0].min()
sn_max = result[:, 0].max()


In [24]:
#plot the result in 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x1.ravel(), y1.ravel(), z1.ravel(), c = 'r', marker = 'o', label = 'Cancer(N =' + str(cnDataNum) +')')
ax.scatter(x2.ravel(), y2.ravel(), z2.ravel(), c = 'b', marker = '^', label = 'Normal(N =' + str(norDataNum) +')')

plt.axis('tight')
plt.title('Specificity: ' + '{0:.3f}'.format(specAvr) +  ' ; ' + 'Sensitivity:' + '{0:.3f}'.format(sensAvr), fontsize = 12)
plt.suptitle('NADH intensity - FAD intensity - Heterogeneit redox', fontsize = 14)
ax.set_xlabel('nadh_intensity')
ax.set_ylabel('fad_intensity')
ax.set_zlabel('redox_hetero')
plt.legend()
plt.show()

In [25]:
print (specAvr, sp_max, sp_min, sensAvr, sn_max, sn_min)

0.9375 1.0 0.666666666667 0.76875 1.0 0.5
